In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class Activation:
    def __init__(self, type):
        self.type = type

    def forward(self, inputs):
        self.inputs = inputs
        if self.type == "relu":
            self.output = np.maximum(0, inputs)
        elif self.type == "sigmoid":
            self.output = 1 / (1 + np.exp(-inputs))
        elif self.type == "linear":
            self.output = inputs
        elif self.type == "tanh":
            self.output = np.tanh(inputs)
        else:
            raise ValueError(f"Invalid activation function type: {self.type}")
        return self.output

    def derivative(self):
        if self.type == "relu":
            return np.where(self.inputs > 0, 1, 0)
        elif self.type == "sigmoid":
            return self.output * (1 - self.output)
        elif self.type == "linear":
            return np.ones_like(self.inputs)
        elif self.type == "tanh":
            return 1 - np.power(self.output, 2)
        else:
            raise ValueError(f"No derivative implemented for activation function type: {self.type}")

class Parameters:
    def __init__(self, input_size, num_neurons):
        self.weights = np.random.randn(input_size, num_neurons) * 0.1
        self.bias = np.random.randn(1, num_neurons) * 0.1

    def get_weights(self):
        return self.weights

    def get_bias(self):
        return self.bias

class Neuron:
    def __init__(self, input_size):
        self.params = Parameters(input_size, 1)  # Each neuron has one output

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.params.get_weights()) + self.params.get_bias()
        return self.output

    def backward(self, d_output):
        if d_output.ndim == 1:
            d_output = d_output[:, np.newaxis]

        self.d_weights = np.dot(self.inputs.T, d_output)
        self.d_bias = np.sum(d_output, axis=0, keepdims=True)
        self.d_inputs = np.dot(d_output, self.params.get_weights().T)

        return self.d_inputs

class Layer:
    def __init__(self, input_size, num_neurons, activation_type):
        self.neurons = [Neuron(input_size) for _ in range(num_neurons)]
        self.activation_fn = Activation(activation_type)

    def forward(self, inputs):
        self.inputs = inputs
        neuron_outputs = np.hstack([neuron.forward(inputs) for neuron in self.neurons])
        return self.activation_fn.forward(neuron_outputs)

    def backward(self, d_output):
        d_output_activation = self.activation_fn.derivative() * d_output
        d_inputs = np.zeros_like(self.inputs, dtype=np.float64)
        for i, neuron in enumerate(self.neurons):
            neuron_d_inputs = neuron.backward(d_output_activation[:, i:i+1])
            d_inputs += neuron_d_inputs
        return d_inputs

class BackPropagation:
    @staticmethod
    def update_parameters(layers, learning_rate):
        for layer in layers:
            for neuron in layer.neurons:
                neuron.params.weights -= learning_rate * neuron.d_weights
                neuron.params.bias -= learning_rate * neuron.d_bias

class LossFunction:
    @staticmethod
    def mse(predicted, actual):
        return np.mean((predicted - actual) ** 2)

    @staticmethod
    def mse_derivative(predicted, actual):
        return 2 * (predicted - actual) / actual.size

class NeuralNetwork:
    def __init__(self):
        self.layers = []

    def add_layer(self, layer):
        self.layers.append(layer)

    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def backward(self, loss_gradient):
        for layer in reversed(self.layers):
            loss_gradient = layer.backward(loss_gradient)

    def train(self, inputs, targets, epochs, learning_rate):
        for epoch in range(epochs):
            outputs = self.forward(inputs)
            loss = LossFunction.mse(outputs, targets)
            print(f"Epoch {epoch+1}, Loss: {loss}")
            loss_gradient = LossFunction.mse_derivative(outputs, targets)
            self.backward(loss_gradient)
            BackPropagation.update_parameters(self.layers, learning_rate)

    def evaluate(self, X, y):
        predictions = self.predict(X)
        predictions = np.round(predictions)
        accuracy = np.mean(predictions == y)
        return accuracy

    def predict(self, inputs):
        return self.forward(inputs)

dataset = pd.read_csv("train_x.csv")
labels = pd.read_csv("train_label.csv")

# Analyzing the shapes of the dataset and labels
dataset_shape = dataset.shape
labels_shape = labels.shape

dataset_shape, labels_shape


from sklearn.model_selection import train_test_split

# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(dataset, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Checking the shapes of the splits
(X_train.shape, y_train.shape), (X_val.shape, y_val.shape), (X_test.shape, y_test.shape)


# Convert the pandas dataframes to numpy arrays for processing with the neural network
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()
X_val_np = X_val.to_numpy()
y_val_np = y_val.to_numpy()
X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()
# Create the neural network
nn = NeuralNetwork()

input_size = 784  
output_size = 10  

# Adding layers to the neural network
nn.add_layer(Layer(input_size=input_size, num_neurons=64, activation_type='relu'))
nn.add_layer(Layer(input_size=64, num_neurons=output_size, activation_type='sigmoid'))

# Training parameters
epochs = 10
learning_rate = 0.1

# Training the neural network
print("Training the Neural Network...")
nn.train(X_train_np, y_train_np, epochs, learning_rate)

# Evaluating the neural network
print("Evaluating on Validation Data...")
val_accuracy = nn.evaluate(X_val_np, y_val_np)
print(f"Validation Accuracy: {val_accuracy}")

print("Evaluating on Test Data...")
test_accuracy = nn.evaluate(X_test_np, y_test_np)
print(f"Test Accuracy: {test_accuracy}")

Training the Neural Network...
Epoch 1, Loss: 0.5186977789322239
Epoch 2, Loss: 0.43074691109074215
Epoch 3, Loss: 0.35225142404690535
Epoch 4, Loss: 0.294664116903738
Epoch 5, Loss: 0.2491645718904508
Epoch 6, Loss: 0.21701506224678652
Epoch 7, Loss: 0.20715692978027314
Epoch 8, Loss: 0.20402803138654085
Epoch 9, Loss: 0.19939696447700178
Epoch 10, Loss: 0.19780999257189022
Evaluating on Validation Data...
Validation Accuracy: 0.795
Evaluating on Test Data...
Test Accuracy: 0.803
